In [1]:
!pip install googletrans==4.0.0-rc1

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import json
import time
from googletrans import Translator

For the dataset "gender_stereotypes_augmented_A_B"

In [4]:
translator = Translator()

In [5]:
input_path = "/home/chakrabort/Documents/Llama2/Dataset/gender_stereotypes_augmented_A_B.json"
output_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/gender_stereotypes_translated_it_with_keys.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Function to translate everything including keys
def translate_all(obj):
    if isinstance(obj, dict):
        new_dict = {}
        for k, v in obj.items():
            try:
                # Translate key
                if k.strip() in ["(A)", "(B)"]:
                    new_key = k
                else:
                    new_key = translator.translate(k, src='en', dest='it').text

                # Translate value
                new_value = translate_all(v)
                new_dict[new_key] = new_value
            except Exception as e:
                print(f"Key translation failed for: {k} → {e}")
                new_dict[k] = translate_all(v)
        return new_dict

    elif isinstance(obj, list):
        return [translate_all(item) for item in obj]

    elif isinstance(obj, str):
        if obj.strip() in ["(A)", "(B)"]:
            return obj
        try:
            translated = translator.translate(obj, src="en", dest="it")
            return translated.text if translated and translated.text else obj
        except Exception as e:
            print(f"Value translation failed for: {obj[:50]}... → {e}")
            return obj
    else:
        return obj

# Perform full translation
translated_data = translate_all(data)

# Save translated file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"✅ Translated everything including keys. Saved to: {output_path}")

✅ Translated everything including keys. Saved to: /home/chakrabort/Documents/Llama2/Dataset_Italian/gender_stereotypes_translated_it_with_keys.json


For the dataset "refusal_data_A_B"

In [20]:
import json
import os

# === CONFIGURATION ===
input_file = "/home/chakrabort/Documents/Llama2/Dataset/refusal_data_A_B.json"  # your full dataset
output_dir = "/home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks"    # folder to save chunks
chunk_size = 30                        # Questions per chunk

# === LOAD DATA ===
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# === SPLIT INTO CHUNKS ===
chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# === CREATE OUTPUT FOLDER ===
os.makedirs(output_dir, exist_ok=True)

# === SAVE EACH CHUNK ===
for i, chunk in enumerate(chunks):
    filename = f"{output_dir}/refusal_data_chunk_{i + 1}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(chunk, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved: {filename} ({len(chunk)} questions)")

print(f"\n✅ All done! {len(chunks)} chunks created in '{output_dir}'")


✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_1.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_2.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_3.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_4.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_5.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_6.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_7.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_8.json (30 questions)
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_9.json (30 que

In [3]:
pip install deep-translator tqdm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import time
import sys
from deep_translator import GoogleTranslator
from tqdm import tqdm

In [12]:
# === CONFIG ===
input_path = "/home/chakrabort/Documents/Llama2/Dataset/refusal_data_chunks/refusal_data_chunk_14.json"
output_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_chunks/refusal_data_14_IT.json"

# === TRANSLATOR ===
translator = GoogleTranslator(source='en', target='it')

# === SAFE TRANSLATE FUNCTION ===
def safe_translate(text, delay=5):
    try:
        time.sleep(delay)
        return translator.translate(text)
    except Exception as e:
        print(f"❌ Translation failed for '{text[:30]}': {e}")
        return text

# === COUNT STRINGS TO TRACK PROGRESS ===
def count_strings(obj):
    if isinstance(obj, dict):
        return sum(count_strings(k) for k in obj.keys()) + sum(count_strings(v) for v in obj.values())
    elif isinstance(obj, list):
        return sum(count_strings(i) for i in obj)
    elif isinstance(obj, str):
        return 1
    return 0

# === TRANSLATE ALL (WITH KEYS + VALUES) ===
def translate_all(obj, pbar=None):
    if isinstance(obj, dict):
        new_dict = {}
        for k, v in obj.items():
            # Translate key
            try:
                new_key = k if k.strip() in ["(A)", "(B)"] else safe_translate(k)
            except Exception as e:
                print(f"⚠️ Key translation failed for: {k} → {e}")
                new_key = k

            # Translate value recursively
            try:
                new_dict[new_key] = translate_all(v, pbar)
            except Exception as e:
                print(f"⚠️ Value translation failed for key '{new_key}' → {e}")
                new_dict[new_key] = v
        return new_dict

    elif isinstance(obj, list):
        return [translate_all(i, pbar) for i in tqdm(obj, desc="📚 Translating list", leave=False, file=sys.stdout)]

    elif isinstance(obj, str):
        if obj.strip() in ["(A)", "(B)"]:
            return obj
        try:
            translated = safe_translate(obj)
            if pbar:
                pbar.update(1)
            return translated
        except Exception as e:
            print(f"⚠️ Value translation failed for: {obj[:40]}... → {e}")
            return obj
    else:
        return obj

# === LOAD INPUT JSON ===
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === COUNT FOR PROGRESS TRACKING ===
total_strings = count_strings(data)
print(f"🔢 Total translatable strings (keys + values): {total_strings}")

# === TRANSLATE WITH PROGRESS BAR ===
with tqdm(total=total_strings, desc="🔁 Translating", unit="item", file=sys.stdout) as pbar:
    translated_data = translate_all(data, pbar)

# === SAVE OUTPUT ===
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Translation complete! Output saved to: {output_path}")

🔢 Total translatable strings (keys + values): 168
🔁 Translating:   0%|          | 0/168 [00:00<?, ?item/s]

🔁 Translating:  17%|█▋        | 28/168 [09:56<49:42, 21.31s/item]

✅ Translation complete! Output saved to: /home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_chunks/refusal_data_14_IT.json


In [14]:
import json
import os
import re

# Folder where chunks are stored
folder_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_chunks"

# Correctly sort files by extracting the number from filename
file_names = sorted(
    [f for f in os.listdir(folder_path) if f.endswith(".json")],
    key=lambda x: int(re.search(r'refusal_data_(\d+)_IT\.json', x).group(1))
)

merged_data = []

# Merge files in proper order
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        chunk_data = json.load(f)
        merged_data.extend(chunk_data)

# Save the merged file
output_path = os.path.join(folder_path, "refusal_data_IT_merged.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"✅ Merged {len(file_names)} chunks correctly into: {output_path}")


✅ Merged 14 chunks correctly into: /home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_chunks/refusal_data_IT_merged.json


In [15]:
import json

# Path to the merged Italian file
input_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_merged.json"
output_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_final.json"

# Load the merged dataset
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Rename the keys in each dictionary
for entry in data:
    if "Risposta_matching_behavior" in entry:
        entry["risposta_correlata"] = entry.pop("Risposta_matching_behavior")
    if "Risposta_not_matching_behavior" in entry:
        entry["risposta_non_correlata"] = entry.pop("Risposta_not_matching_behavior")

# Save the updated dataset
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ Keys updated and saved to: {output_path}")


✅ Keys updated and saved to: /home/chakrabort/Documents/Llama2/Dataset_Italian/refusal_data_IT_final.json


For the dataset "stereotype_data_A_B_subset"

In [3]:
# Dividing into seperate jsons

import json
import os

# Path to input JSON file
input_path = "/home/chakrabort/Documents/Llama2/Dataset/stereotype_data_A_B_subset.json"

# Output directory
output_dir = "/home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks"

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Load the full JSON data
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Save each top-level key's content to its own file
for key in data:
    output_path = os.path.join(output_dir, f"{key}.json")
    with open(output_path, "w", encoding="utf-8") as out_file:
        json.dump(data[key], out_file, ensure_ascii=False, indent=2)
    print(f"✅ Saved: {output_path}")


✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/race.json
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/gender.json
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/profession.json
✅ Saved: /home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/religion.json


In [1]:
pip install deep-translator tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import time
import sys
from deep_translator import GoogleTranslator
from tqdm import tqdm

In [5]:
# === CONFIG ===
input_path = "/home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/profession_chunks/profession_chunk_4.json"
output_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/profession_IT_chunks/profession_IT_chunk_4.json"

# === TRANSLATOR ===
translator = GoogleTranslator(source='en', target='it')

# === SAFE TRANSLATE FUNCTION ===
def safe_translate(text, delay=5):
    try:
        time.sleep(delay)
        return translator.translate(text)
    except Exception as e:
        print(f"❌ Translation failed for '{text[:30]}': {e}")
        return text

# === COUNT STRINGS TO TRACK PROGRESS ===
def count_strings(obj):
    if isinstance(obj, dict):
        return sum(count_strings(k) for k in obj.keys()) + sum(count_strings(v) for v in obj.values())
    elif isinstance(obj, list):
        return sum(count_strings(i) for i in obj)
    elif isinstance(obj, str):
        return 1
    return 0

# === TRANSLATE ALL (WITH KEYS + VALUES) ===
def translate_all(obj, pbar=None):
    if isinstance(obj, dict):
        new_dict = {}
        for k, v in obj.items():
            # Translate key
            try:
                new_key = k if k.strip() in ["(A)", "(B)"] else safe_translate(k)
            except Exception as e:
                print(f"⚠️ Key translation failed for: {k} → {e}")
                new_key = k

            # Translate value recursively
            try:
                new_dict[new_key] = translate_all(v, pbar)
            except Exception as e:
                print(f"⚠️ Value translation failed for key '{new_key}' → {e}")
                new_dict[new_key] = v
        return new_dict

    elif isinstance(obj, list):
        return [translate_all(i, pbar) for i in tqdm(obj, desc="📚 Translating list", leave=False, file=sys.stdout)]

    elif isinstance(obj, str):
        if obj.strip() in ["(A)", "(B)"]:
            return obj
        try:
            translated = safe_translate(obj)
            if pbar:
                pbar.update(1)
            return translated
        except Exception as e:
            print(f"⚠️ Value translation failed for: {obj[:40]}... → {e}")
            return obj
    else:
        return obj

# === LOAD INPUT JSON ===
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === COUNT FOR PROGRESS TRACKING ===
total_strings = count_strings(data)
print(f"🔢 Total translatable strings (keys + values): {total_strings}")

# === TRANSLATE WITH PROGRESS BAR ===
with tqdm(total=total_strings, desc="🔁 Translating", unit="item", file=sys.stdout) as pbar:
    translated_data = translate_all(data, pbar)

# === SAVE OUTPUT ===
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Translation complete! Output saved to: {output_path}")

🔢 Total translatable strings (keys + values): 1200
🔁 Translating:  17%|█▋        | 200/1200 [1:10:25<5:52:09, 21.13s/item]

✅ Translation complete! Output saved to: /home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/profession_IT_chunks/profession_IT_chunk_4.json


In [5]:
import json
import os

# === CONFIGURATION ===
input_path = "/home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/profession.json"  # replace with your file path
output_dir = "/home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/profession_chunks"         # replace with your output directory
chunk_size = 200

# === CREATE OUTPUT DIRECTORY IF NOT EXISTS ===
os.makedirs(output_dir, exist_ok=True)

# === LOAD JSON FILE ===
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === SPLIT INTO CHUNKS AND SAVE ===
for i in range(0, len(data), chunk_size):
    chunk = data[i:i + chunk_size]
    chunk_filename = f"profession_chunk_{i // chunk_size + 1}.json"
    chunk_path = os.path.join(output_dir, chunk_filename)
    with open(chunk_path, "w", encoding="utf-8") as f:
        json.dump(chunk, f, ensure_ascii=False, indent=2)

print(f"✅ Done! Created {len(data) // chunk_size + (1 if len(data) % chunk_size else 0)} chunk files in {output_dir}")


✅ Done! Created 5 chunk files in /home/chakrabort/Documents/Llama2/Dataset/stereotype_data_chunks/profession_chunks


In [1]:
import json
import os

# === CONFIGURATION ===
folder_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/profession_IT_chunks"  # change this to your actual path
output_path = os.path.join(folder_path, "profession_IT.json")

# === FILE LIST IN ORDER ===
file_names = [
    "profession_IT_chunk_1.json",
    "profession_IT_chunk_2.json",
    "profession_IT_chunk_3.json",
    "profession_IT_chunk_4.json",
    "profession_IT_chunk_5.json"
]

# === MERGE ===
merged_data = []

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        chunk = json.load(f)
        merged_data.extend(chunk)

# === SAVE MERGED FILE ===
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"✅ Merged {len(file_names)} chunks. Total entries: {len(merged_data)}")
print(f"📄 Saved to: {output_path}")


✅ Merged 5 chunks. Total entries: 827
📄 Saved to: /home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/profession_IT_chunks/profession_IT.json


In [2]:
import json

# Load each JSON file from the specified directory
with open("/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/race_IT.json", "r", encoding="utf-8") as f:
    race_data = json.load(f)

with open("/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/gender_IT.json", "r", encoding="utf-8") as f:
    gender_data = json.load(f)

with open("/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/profession_IT.json", "r", encoding="utf-8") as f:
    profession_data = json.load(f)

with open("/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT_chunks/religion_IT.json", "r", encoding="utf-8") as f:
    religion_data = json.load(f)

# Merge into a single dictionary with Italian keys
merged_data = {
    "razza": race_data,
    "genere": gender_data,
    "professione": profession_data,
    "religione": religion_data
}

# Save the merged JSON
output_path = "/home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"✅ File unificato salvato come: {output_path}")


✅ File unificato salvato come: /home/chakrabort/Documents/Llama2/Dataset_Italian/stereotype_data_IT.json
